In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    	builder.\
    	appName("จาก raw data สู่ Disk").\
    	master("spark://spark-master:7077").\
    	config("spark.executor.memory", "1000m").\
    	config("spark.executor.cores", "1").\
    	config("spark.cores.max", "4").\
    	getOrCreate()

23/06/13 02:15:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
colors = ['white','green','yellow','red','brown','pink'] 

In [4]:
type(colors)

list

In [5]:
colors[0]

'white'

In [6]:
type(colors[0])

str

In [8]:
color_rdd = spark.sparkContext.parallelize(colors)

In [9]:
type(color_rdd)

pyspark.rdd.RDD

In [10]:
color_rdd.collect()

['white', 'green', 'yellow', 'red', 'brown', 'pink']

In [11]:
type(color_rdd.collect())

list

In [12]:
color_rdd.map(lambda x:(x,len(x))).collect()

[('white', 5),
 ('green', 5),
 ('yellow', 6),
 ('red', 3),
 ('brown', 5),
 ('pink', 4)]

In [13]:
type(color_rdd.map(lambda x:(x,len(x))).collect())

list

In [14]:
color_rdd.map(lambda x:(x,len(x))).collect()[3]

('red', 3)

In [15]:
type(color_rdd.map(lambda x:(x,len(x))).collect()[3])

tuple

In [16]:
keyval_rdd = color_rdd.map(lambda x:(x,len(x)))

In [17]:
keyval_rdd.collect()

[('white', 5),
 ('green', 5),
 ('yellow', 6),
 ('red', 3),
 ('brown', 5),
 ('pink', 4)]

In [18]:
type(keyval_rdd)

pyspark.rdd.PipelinedRDD

In [19]:
keyval_rdd.toDF(['color','length']).printSchema()

root
 |-- color: string (nullable = true)
 |-- length: long (nullable = true)



In [20]:
keyval_rdd.collect()

[('white', 5),
 ('green', 5),
 ('yellow', 6),
 ('red', 3),
 ('brown', 5),
 ('pink', 4)]

In [21]:
color_df = keyval_rdd.toDF(['color','length'])

In [22]:
color_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



In [24]:
color_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- length: long (nullable = true)



In [25]:
color_df

DataFrame[color: string, length: bigint]

In [26]:
color_df.count()

6

In [27]:
color_df.show(100)

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



In [28]:
color_df.collect()

[Row(color='white', length=5),
 Row(color='green', length=5),
 Row(color='yellow', length=6),
 Row(color='red', length=3),
 Row(color='brown', length=5),
 Row(color='pink', length=4)]

In [30]:
color_df.columns

['color', 'length']

In [31]:
color_df.drop('length').show()

+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
| brown|
|  pink|
+------+



In [32]:
color_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



In [33]:
onlyColorName_df = color_df.drop('length')

In [34]:
onlyColorName_df.show()

+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
| brown|
|  pink|
+------+



In [35]:
color_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



In [36]:
from pyspark.sql import functions as sparkf

In [37]:
color_df.filter(sparkf.col('length').between(4,5))\
.select(sparkf.col('color').alias('mid_length')).show()

+----------+
|mid_length|
+----------+
|     white|
|     green|
|     brown|
|      pink|
+----------+



In [39]:
from pyspark.sql import functions as sparkf

In [40]:
sparkf

<module 'pyspark.sql.functions' from '/usr/local/lib/python3.9/dist-packages/pyspark/sql/functions.py'>

In [41]:
color_df.filter((sparkf.col('length') > 4)&(sparkf.col('color') !="white"))\
.show()

+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
| brown|     5|
+------+------+



In [45]:
color_df.orderBy(sparkf.col('length').desc(), sparkf.col('color').asc()).show()

+------+------+
| color|length|
+------+------+
|yellow|     6|
| brown|     5|
| green|     5|
| white|     5|
|  pink|     4|
|   red|     3|
+------+------+



In [46]:
color_df.groupBy('length').count().show()

+------+-----+
|length|count|
+------+-----+
|     6|    1|
|     5|    3|
|     3|    1|
|     4|    1|
+------+-----+



In [48]:
color_df.groupBy('color').agg(sparkf.avg('length')).show()

+------+-----------+
| color|avg(length)|
+------+-----------+
| green|        5.0|
|yellow|        6.0|
| white|        5.0|
|  pink|        4.0|
|   red|        3.0|
| brown|        5.0|
+------+-----------+



In [49]:
color_df.describe().show()

+-------+------+------------------+
|summary| color|            length|
+-------+------+------------------+
|  count|     6|                 6|
|   mean|  null| 4.666666666666667|
| stddev|  null|1.0327955589886444|
|    min| brown|                 3|
|    max|yellow|                 6|
+-------+------+------------------+



In [50]:
color_df.count()

6